In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder

In [4]:
df=pd.read_csv('../data/data2022.csv')
print(df.columns)

Index(['State', 'Sex', 'GeneralHealth', 'PhysicalHealthDays',
       'MentalHealthDays', 'LastCheckupTime', 'PhysicalActivities',
       'SleepHours', 'RemovedTeeth', 'HadHeartAttack', 'HadAngina',
       'HadStroke', 'HadAsthma', 'HadSkinCancer', 'HadCOPD',
       'HadDepressiveDisorder', 'HadKidneyDisease', 'HadArthritis',
       'HadDiabetes', 'DeafOrHardOfHearing', 'BlindOrVisionDifficulty',
       'DifficultyConcentrating', 'DifficultyWalking',
       'DifficultyDressingBathing', 'DifficultyErrands', 'SmokerStatus',
       'ECigaretteUsage', 'ChestScan', 'RaceEthnicityCategory', 'AgeCategory',
       'HeightInMeters', 'WeightInKilograms', 'BMI', 'AlcoholDrinkers',
       'HIVTesting', 'FluVaxLast12', 'PneumoVaxEver', 'TetanusLast10Tdap',
       'HighRiskLastYear', 'CovidPos'],
      dtype='object')


In [5]:
df=pd.read_csv('../data/data2022.csv')
df.head()
'''
columns_to_keep = [
    'HeartDisease','BMI', 'Smoking', 'Stroke', 'PhysicalHealthDays',
    'DiffWalking', 'Sex', 'AgeCategory', 'Diabetic',
    'Asthma', 'KidneyDisease'
]

df = df[columns_to_keep]
'''

"\ncolumns_to_keep = [\n    'HeartDisease','BMI', 'Smoking', 'Stroke', 'PhysicalHealthDays',\n    'DiffWalking', 'Sex', 'AgeCategory', 'Diabetic',\n    'Asthma', 'KidneyDisease'\n]\n\ndf = df[columns_to_keep]\n"

In [6]:
df1 = df.copy()
le=LabelEncoder()
categorical = df.select_dtypes(include = 'O')
categorical.columns
for feature in categorical:
    df1[feature]=le.fit_transform(df1[feature])
    print(feature)

State
Sex
GeneralHealth
LastCheckupTime
PhysicalActivities
RemovedTeeth
HadHeartAttack
HadAngina
HadStroke
HadAsthma
HadSkinCancer
HadCOPD
HadDepressiveDisorder
HadKidneyDisease
HadArthritis
HadDiabetes
DeafOrHardOfHearing
BlindOrVisionDifficulty
DifficultyConcentrating
DifficultyWalking
DifficultyDressingBathing
DifficultyErrands
SmokerStatus
ECigaretteUsage
ChestScan
RaceEthnicityCategory
AgeCategory
AlcoholDrinkers
HIVTesting
FluVaxLast12
PneumoVaxEver
TetanusLast10Tdap
HighRiskLastYear
CovidPos


In [10]:
def split_data(X, Y, feature):
    # X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2,stratify=Y)
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2,stratify=df1[feature])
    return X_train, X_test, y_train, y_test
def over_sample_date(X_train, y_train, data_rate = 1):
    minority_class = y_train.value_counts().idxmin()    # 1
    majority_class = y_train.value_counts().idxmax()    # 0

    minority_data = X_train[y_train == minority_class]
    majority_data = X_train[y_train == majority_class]
    minority_labels = y_train[y_train == minority_class]
    majority_labels = y_train[y_train == majority_class]

    oversampled_minority_data = minority_data.sample(int(len(majority_data)*data_rate), replace=True)
    oversampled_minority_labels = minority_labels.sample(int(len(majority_data)*data_rate), replace=True)
    X_train_oversampled = pd.concat([majority_data, oversampled_minority_data])
    y_train_oversampled = pd.concat([majority_labels, oversampled_minority_labels])
    return X_train_oversampled, y_train_oversampled

## Change data_rate and other things to improve the data

In [43]:
# get the training dataset and test dataset
X=df1.drop('HadHeartAttack',axis=1)
Y=df1['HadHeartAttack']
X_train, X_test, y_train, y_test = split_data(X,Y, "HadHeartAttack")
X_train_oversampled, Y_train_oversampled = over_sample_date(X_train, y_train, data_rate = 1)

In [44]:
# transform the training data
scaler = StandardScaler()
X_train_oversampled_T = scaler.fit_transform(X_train_oversampled)
X_test_T = scaler.transform(X_test)

In [45]:
log_reg = LogisticRegression()
log_reg.fit(X_train_oversampled_T, Y_train_oversampled)

y_pred = log_reg.predict(X_test_T)
#acc=log_reg.score(y_test,y_pred)
print(classification_report(y_test, y_pred))
confusion_matrix(y_test,y_pred)

              precision    recall  f1-score   support

           0       0.98      0.83      0.90     46518
           1       0.21      0.77      0.33      2687

    accuracy                           0.83     49205
   macro avg       0.60      0.80      0.62     49205
weighted avg       0.94      0.83      0.87     49205



array([[38747,  7771],
       [  623,  2064]])

In [46]:
log_reg.coef_,log_reg.intercept_

(array([[-0.00136227,  0.37913133, -0.03444018,  0.0871154 ,  0.03910174,
          0.11544806, -0.04395245, -0.05543116, -0.06759481,  1.17076279,
          0.33339695,  0.03266067, -0.02204017,  0.06829962,  0.02989891,
          0.06831557,  0.0286537 ,  0.18987811,  0.05294126,  0.06515129,
          0.04416603,  0.09895978, -0.03410858,  0.0588833 , -0.21117684,
          0.03572442,  0.34976337, -0.00572757,  0.69643557, -0.1476619 ,
          0.17557901, -0.11312748, -0.13325854, -0.00568753, -0.0787146 ,
          0.02571391, -0.01498358,  0.02436731,  0.02283218]]),
 array([0.16600727]))

In [47]:
from sklearn.preprocessing import StandardScaler
import pickle
def normalize_row(row, scaler):
    # Reshape row to match expected input of scaler
    row_reshaped = row.values.reshape(1, -1)
    # Apply the same transformation as training data
    normalized_row = scaler.transform(row_reshaped)
    return normalized_row  # Flatten to match the shape of the original row

# Fit scaler
scaler = StandardScaler().fit(X_train_oversampled.values)
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
df_new = X_train_oversampled.iloc[0]

df_new_normalized = normalize_row(df_new, scaler)

print(df_new_normalized)

# If true, this proof the correctness of normalization
print(X_train_oversampled_T[0] == df_new_normalized)


[[ 1.00448853 -1.10945754 -0.16499011 -0.51520952 -0.18001931  0.37420921
   0.64011494 -0.63060109 -0.48286087  1.65271139 -0.35163785  2.24666072
   2.83439435  2.42463054 -0.53795208 -0.31438525  1.10011476  1.70584444
  -0.40179989 -0.2973369  -0.39932005 -0.59019985 -0.2589463   2.83053256
   0.77702643 -0.46208217  0.84454938 -0.29817373  0.3453844  -0.27890557
   1.10014192  1.48333217 -0.95344425  1.43081769  0.85504849  0.95737533
  -0.14575936 -0.18983213  1.61139995]]
[[ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True]]


In [16]:
print(list(X_train_oversampled.iloc[0]))

[26.63, 1.0, 0.0, 0.0, 0.0, 1.0, 10.0, 0.0, 0.0, 0.0]


In [19]:
# Use user input to get 
import numpy as np

user_input = [29.76, 0.0, 0.0, 0.0, 0.0, 1.0, 7.0, 0.0, 0.0, 0.0]
with open('scaler.pkl', 'rb') as f:
    scaler2 = pickle.load(f)
user_input_T = normalize_row(pd.DataFrame([user_input]),scaler2)

coef = np.array([0.09889642, 0.22637016, 0.30912101, 0.2488255 , 0.18334112,
         0.37937364, 1.03940385, 0.25440507, 0.13933909, 0.17579232]).flatten()
intercept = -1.29850855

# Compute the linear combination of inputs and weights
z = np.dot(user_input_T, coef) + intercept

# Apply the logistic function
p = 1 / (1 + np.exp(-z))

print("The predicted probability is:", p)


The predicted probability is: [0.16067047]


In [21]:
col = (list(df.columns[1:]))
coef = (list(log_reg.coef_[0]))
print(len(col), len(coef))
num = 0;
for i in range(len(col)):
    if(coef[i] > 0.08 or coef[i] < -0.08):
        print(col[i], coef[i])
        num+=1
print(num)

39 39
GeneralHealth 0.36402090284437594
PhysicalActivities 0.10137818621197459
HadAngina 0.9737636569231289
HadStroke 0.27964870799377695
HadDiabetes 0.17573292365639212
DifficultyWalking 0.08256261882192018
SmokerStatus -0.19966559729113295
ChestScan 0.3410593024476512
AgeCategory 0.6594176125386868
AlcoholDrinkers -0.12724874632043026
FluVaxLast12 -0.08333360627771243
11


In [49]:
columns = list(df.columns[1:])
coefficients = list(log_reg.coef_[0])

features_with_coef = list(zip(columns, coefficients))

sorted_features = sorted(features_with_coef, key=lambda x: abs(x[1]), reverse=True)

top_features = sorted_features[:10]

for feature, coef in top_features:
    print(feature)
print(len(top_features))


HadAngina
AgeCategory
GeneralHealth
ChestScan
HadStroke
SmokerStatus
HadDiabetes
WeightInKilograms
HeightInMeters
AlcoholDrinkers
10


In [ ]:
feature_list = [
    "HadAngina",
    "AgeCategory",
    "GeneralHealth",
    "ChestScan",
    "HadStroke",
    "SmokerStatus",
    "HadDiabetes",
    "WeightInKilograms",
    "HeightInMeters",
    "AlcoholDrinkers"
]

In [53]:
import pickle
from joblib import dump, load
df=pd.read_csv('../data/data2022.csv')
df.head()

feature_list = [
    "HadHeartAttack",
    "HadAngina",
    "AgeCategory",
    "GeneralHealth",
    "ChestScan",
    "HadStroke",
    "SmokerStatus",
    "HadDiabetes",
    "WeightInKilograms",
    "HeightInMeters",
    "AlcoholDrinkers"
]

df = df[feature_list]
df1 = df.copy()
le=LabelEncoder()
categorical = df.select_dtypes(include = 'O')
categorical.columns
for feature in categorical:
    df1[feature]=le.fit_transform(df1[feature])

X=df1.drop('HadHeartAttack',axis=1)
Y=df1['HadHeartAttack']
X_train, X_test, y_train, y_test = split_data(X,Y, "HadHeartAttack")
X_train_oversampled, Y_train_oversampled = over_sample_date(X_train, y_train, data_rate = 1)
scaler = StandardScaler()
X_train_oversampled_T = scaler.fit_transform(X_train_oversampled)
X_test_T = scaler.transform(X_test)

log_reg = LogisticRegression()
log_reg.fit(X_train_oversampled_T, Y_train_oversampled)

y_pred = log_reg.predict(X_test_T)

print(classification_report(y_test, y_pred))
confusion_matrix(y_test,y_pred)

print(log_reg.coef_,log_reg.intercept_)

scaler = StandardScaler().fit(X_train_oversampled.values)
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

dump(log_reg, 'logistic_model.joblib')


              precision    recall  f1-score   support

           0       0.98      0.83      0.90     46518
           1       0.20      0.74      0.32      2687

    accuracy                           0.83     49205
   macro avg       0.59      0.79      0.61     49205
weighted avg       0.94      0.83      0.87     49205

[[ 1.22202847  0.70660896 -0.08346049  0.42422738  0.38554414 -0.29696647
   0.24849091  0.11139583  0.08505041 -0.17312089]] [0.16660101]


['logistic_model.joblib']

In [112]:
# Load the model back from the file
loaded_model = load('logistic_model.joblib')

# Use loaded_model to predict or evaluate to ensure it works
predictions = loaded_model.predict_proba(X_test_T[20:31])
print(predictions[:])

[[0.56895911 0.43104089]
 [0.26644386 0.73355614]
 [0.19972537 0.80027463]
 [0.01183231 0.98816769]
 [0.94912316 0.05087684]
 [0.35119981 0.64880019]
 [0.53187845 0.46812155]
 [0.90900687 0.09099313]
 [0.25803047 0.74196953]
 [0.76692745 0.23307255]
 [0.09996726 0.90003274]]


In [83]:
for i in range(len(X_test)):
    if y_test.iloc[i] == 1:
        print(i)

21
27
30
73
78
86
91
108
120
145
187
194
227
229
238
269
286
287
303
327
400
401
405
412
416
425
428
438
520
552
555
558
590
620
634
637
645
664
698
700
726
731
735
739
753
765
771
792
801
802
812
834
844
850
865
892
911
960
978
1032
1050
1054
1079
1100
1108
1114
1121
1122
1156
1161
1163
1165
1181
1183
1185
1207
1222
1224
1228
1242
1247
1360
1375
1397
1481
1496
1555
1588
1605
1640
1671
1676
1688
1714
1716
1729
1772
1773
1796
1813
1824
1850
1931
1951
1981
2009
2036
2045
2048
2053
2079
2097
2102
2191
2200
2259
2260
2299
2300
2311
2315
2356
2357
2369
2370
2383
2386
2389
2404
2419
2421
2438
2498
2504
2509
2515
2516
2526
2544
2551
2572
2607
2610
2631
2667
2694
2739
2756
2851
2858
2878
2887
2914
2919
2946
2953
2963
2986
3006
3011
3032
3043
3044
3052
3057
3072
3106
3119
3121
3122
3216
3217
3238
3264
3274
3304
3319
3350
3352
3368
3375
3376
3397
3398
3413
3422
3432
3448
3470
3514
3533
3551
3552
3565
3592
3601
3605
3615
3638
3643
3648
3669
3684
3707
3722
3766
3777
3778
3799
3831
3840
3851
3856
3

In [119]:
list(X_test.iloc[0])

[0.0, 9.0, 4.0, 1.0, 0.0, 3.0, 0.0, 111.13, 1.63, 0.0]